### Shapash Model Overview
https://shapash.readthedocs.io/en/latest/

##### With this tutorial you:
Understand how to create a Shapash SmartPredictor to make prediction and have local explanation in production with a simple use case.

This tutorial describes the different steps from training the model to Shapash SmartPredictor deployment. A more detailed tutorial allows you to know more about the SmartPredictor Object.

Contents:

- Build a Regressor
- Compile Shapash SmartExplainer
- From Shapash SmartExplainer to SmartPredictor
- Save Shapash Smartpredictor Object in pickle file
- Make a prediction

In [1]:
import seaborn as sns

In [2]:
df=sns.load_dataset('tips')

In [3]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [4]:
### Divide the dataset into independent and dependent dataset
y=df['tip']
X=df[df.columns.difference(['tip'])]

In [5]:
X.head()

,day,sex,size,smoker,time,total_bill
0,Sun,Female,2,No,Dinner,16.99
1,Sun,Male,3,No,Dinner,10.34
2,Sun,Male,3,No,Dinner,21.01
3,Sun,Male,2,No,Dinner,23.68
4,Sun,Female,4,No,Dinner,24.59


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.3 KB


In [7]:
X['day']=X['day'].cat.codes
X['sex']=X['sex'].cat.codes
X['smoker']=X['smoker'].cat.codes
X['time']=X['time'].cat.codes

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [8]:
X

,day,sex,size,smoker,time,total_bill
0,3,1,2,1,1,16.99
1,3,0,3,1,1,10.34
2,3,0,3,1,1,21.01
3,3,0,2,1,1,23.68
4,3,1,4,1,1,24.59
...,...,...,...,...,...,...
239,2,0,3,1,1,29.03
240,2,1,2,0,1,27.18
241,2,0,2,0,1,22.67
242,2,0,2,1,1,17.82


In [9]:
### Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=0.75,random_state=1)

In [10]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=200).fit(X_train,y_train)

#### Lets Understand Our Model With Shapash 
In this section, we use the SmartExplainer Object from shapash.

- It allows users to understand how the model works with the specified data.
- This object must be used only for data mining step. Shapash provides another object for deployment.


In [11]:
!pip install shapash

     |████████████████████████████████| 870kB 21.0MB/s 
     |████████████████████████████████| 102kB 8.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 552kB 36.7MB/s 
     |████████████████████████████████| 1.0MB 47.4MB/s 
     |████████████████████████████████| 327kB 28.3MB/s 
     |████████████████████████████████| 645kB 41.2MB/s 
     |████████████████████████████████| 3.5MB 29.2MB/s 
     |████████████████████████████████| 194kB 50.8MB/s 
     |████████████████████████████████| 13.1MB 300kB/s 
     |████████████████████████████████| 1.8MB 39.8MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
     |████████████████████████████████| 133kB 50.6MB/s 
     |████████████████████████████████| 122kB 50.7MB/s 
     |████████████████████████████████| 747kB 51.7MB/s 
     |████████████████████████████████| 727kB 45.6MB/s 
     |███████████████████████████████

In [12]:
from shapash.explainer.smart_explainer import SmartExplainer


In [13]:
xpl = SmartExplainer()

In [14]:
xpl.compile(
    x=X_test,
    model=regressor,
   
)

Backend: Shap TreeExplainer


In [15]:
xpl

#### Lets Understand the results of your trained model
Then, we can easily get a first summary of the explanation of the model results.

- Here, we chose to get the 3 most contributive features for each prediction.
- We used a wording to get features names more understandable in operationnal case.

In [18]:
app = xpl.run_app(title_story='Tips Dataset')

Dash is running on http://0.0.0.0:8050/



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:1490: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

INFO:root:Your Shapash application run on http://8459d193c2ec:8050/


Dash is running on http://0.0.0.0:8050/



INFO:root:Use the method .kill() to down your app.
INFO:shapash.webapp.smart_app:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app "shapash.webapp.smart_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.7/dist-packages/shapash/utils/threading.py", line 29, in __run
    self.__run_backup()
  File "/usr/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/shapash/explainer/smart_explainer.py", line 920, in <lambda>
    target=lambda: self.smartapp.app.run_server(debug=False, host=host, port=port))
  File "/usr/local/lib/python3.7/dist-packages/dash/dash.py", line 1716, in run_server
    self.server.run(host=host, port=port, debug=debug, **flask_run_options)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 1052, in run_simple
    inner()
  File "/usr/local/lib/python3.7/dist-pa

In [19]:
predictor = xpl.to_smartpredictor()

AttributeError: ignored

In [20]:
predictor.save('./predictor.pkl')

NameError: ignored

In [21]:
from shapash.utils.load_smartpredictor import load_smartpredictor
predictor_load = load_smartpredictor('./predictor.pkl')

FileNotFoundError: ignored

#### Make a prediction with your SmartPredictor
In order to make new predictions and summarize local explainability of your model on new datasets, you can use the method add_input of the SmartPredictor.

- The add_input method is the first step to add a dataset for prediction and explainability.
- It checks the structure of the dataset, the prediction and the contribution if specified.
- It applies the preprocessing specified in the initialisation and reorder the features with the order used by the model. (see the documentation of this method)
- In API mode, this method can handle dictionnaries data which can be received from a GET or a POST request.
- Add data
- The x input in add_input method doesn't have to be encoded, add_input applies preprocessing.

In [ ]:
predictor_load.add_input(x=X, ypred=y)

In [ ]:
detailed_contributions = predictor_load.detail_contributions()


In [ ]:
detailed_contributions.head()

,tip,day,sex,size,smoker,time,total_bill
0,1.01,0.058890,0.107123,-0.021913,-0.032742,-0.005985,0.183477
1,1.66,0.083062,-0.058770,0.003739,-0.062743,-0.018027,-1.230111
2,3.50,0.084890,-0.021616,-0.005301,-0.008053,0.006988,0.428840
3,3.31,0.125816,-0.021922,-0.043787,0.022929,0.007135,0.074177
4,3.61,0.048222,0.058356,-0.053819,-0.006406,-0.006721,-0.319232


#### Summarize explanability of the predictions
- You can use the summarize method to summarize your local explainability
- This summary can be configured with modify_mask method so that you have explainability that meets your operational needs.

In [ ]:

predictor_load.modify_mask(max_contrib=3)

In [ ]:
explanation = predictor_load.summarize()

In [ ]:
explanation.head()

,tip,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
0,1.01,total_bill,16.99,0.183477,sex,1.0,0.107123,day,3.0,0.05889
1,1.66,total_bill,10.34,-1.230111,day,3.0,0.083062,smoker,1.0,-0.062743
2,3.50,total_bill,21.01,0.42884,day,3.0,0.08489,sex,0.0,-0.021616
3,3.31,day,3.0,0.125816,total_bill,23.68,0.074177,size,2.0,-0.043787
4,3.61,total_bill,24.59,-0.319232,sex,1.0,0.058356,size,4.0,-0.053819
